In [46]:
#import dependencies
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import Sequential
!pip install keras_tuner
import keras_tuner as kt
%matplotlib inline

# Dependencies From Christopher
#import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install hvplot

In [ ]:
def examin_data():
  # Read the CSV file into a Pandas DataFrame
  cvd_df=pd.read_csv("Framingham_clean.csv")

  # Review the DataFrame
  print("\nFirst 5 rows of CVS DataFrame.\n")
  print(cvd_df.head())

  print("\nInformation about DataFrame data types.\n")
  print(cvd_df.info())

  print("\nLooking for Null Values in DataFrame.\n")
  print(cvd_df.isnull().sum())

  print("\nWhat is the shape of the DataFrame.\n")
  print(cvd_df.shape)

  print("\nWhat is the Statistical information about the DataFrame.\n")
  print(cvd_df.describe())

  print("\nWhat are the columns of the DataFrame.\n")
  print(cvd_df.columns)

In [ ]:
# Read the CSV file into a Pandas DataFrame
cvd_df=pd.read_csv("Framingham_clean.csv")
examin_data()


First 5 rows of CVS DataFrame.

   Sex_Male  Age  Education  Current_Smoker  Cigarettes_Per_Day  \
0         1   39          4               0                   0   
1         0   46          2               0                   0   
2         1   48          1               1                  20   
3         0   61          3               1                  30   
4         0   46          3               1                  23   

   Blood_Pressure_Medications  Prevalent_Stroke  Prevalent_Hypertension  \
0                           0                 0                       0   
1                           0                 0                       0   
2                           0                 0                       0   
3                           0                 0                       1   
4                           0                 0                       0   

   Diabetes  Total_Cholesterol  Systolic_Blood_Pressure  \
0         0                195                    106.

In [ ]:
# Defining the Features
X = cvd_df.copy()
X.drop("TenYearCHD", axis=1, inplace=True)
X.head()

,Sex_Male,Age,Education,Current_Smoker,Cigarettes_Per_Day,Blood_Pressure_Medications,Prevalent_Stroke,Prevalent_Hypertension,Diabetes,Total_Cholesterol,Systolic_Blood_Pressure,Diastolic_Blood_Pressure,BMI,Heart_Rate,Glucose
0,1,39,4,0,0,0,0,0,0,195,106.0,70.0,26.97,80,77
1,0,46,2,0,0,0,0,0,0,250,121.0,81.0,28.73,95,76
2,1,48,1,1,20,0,0,0,0,245,127.5,80.0,25.34,75,70
3,0,61,3,1,30,0,0,1,0,225,150.0,95.0,28.58,65,103
4,0,46,3,1,23,0,0,0,0,285,130.0,84.0,23.10,85,85


In [ ]:
# Defining target
y = cvd_df["TenYearCHD"].ravel()
y[:5]

array([0, 0, 0, 1, 0])

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Using StandardScaler() module to normalize the data and Fitting the data
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)


In [ ]:
# Seeking the best n_component value for the PCA
# Starting n_components value with 3 
pca_value = 3
pca_sum = 0

while pca_sum < .99:

  pca = PCA(n_components = pca_value)

  # Fitting data to PCA model
  X_train_scale_pca = pca.fit_transform(X_train_scaled)
  X_test_scale_pca = pca.fit_transform(X_test_scaled)

  # Adding up the total variance
  pca_sum = pca.explained_variance_ratio_.sum()
  pca_value = pca_value + 1

  print("Total explained variance:", pca_sum, "\nPCA value:", pca_value)

Total explained variance: 0.4496870846335056 
PCA value: 4
Total explained variance: 0.529496688768673 
PCA value: 5
Total explained variance: 0.6029679188177296 
PCA value: 6
Total explained variance: 0.6733985007662893 
PCA value: 7
Total explained variance: 0.7414328221380071 
PCA value: 8
Total explained variance: 0.7972291547145793 
PCA value: 9
Total explained variance: 0.8474844349396998 
PCA value: 10
Total explained variance: 0.8931556761323813 
PCA value: 11
Total explained variance: 0.9293786535963177 
PCA value: 12
Total explained variance: 0.954522027853017 
PCA value: 13
Total explained variance: 0.9745660417568123 
PCA value: 14
Total explained variance: 0.98879897183688 
PCA value: 15
Total explained variance: 1.0 
PCA value: 16


In [14]:
pca = PCA(n_components = 7)

# Fitting data to PCA model
X_train_scale_pca = pca.fit_transform(X_train_scaled)
X_test_scale_pca = pca.fit_transform(X_test_scaled)

pca.explained_variance_ratio_

array([0.21332061, 0.12738815, 0.10897833, 0.0798096 , 0.07347123,
       0.07043058, 0.06803432])

In [34]:
def matrix_confusion(y_test, prediction,label):
  # Calculating the confusion matrix
  cm = confusion_matrix(y_test, prediction)
  cm_df = pd.DataFrame(
      cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
  )

  # Calculating the accuracy score
  acc_score = accuracy_score(y_test, prediction)

  # Displaying results
  print(label)
  display(cm_df)
  print(f"RF Accuracy Score : {acc_score}")
  print("RF Classification Report")
  print(classification_report(y_test, prediction))

In [50]:
# Creating a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allowing kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allowing kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=7))

    # Allowing kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [51]:
def random_forest_pca(X_train_scale_pca, y_train, X_test_scale_pca, y_test, prediction):
  label = "Random Forest PCA"
  # Creating a random forest classifier
  rf_model = RandomForestClassifier(n_estimators=500, random_state=1).fit(X_train_scale_pca, y_train)
  # Making Prediction using the testing data
  prediction = rf_model.predict(X_test_scale_pca)
  matrix_confusion(y_test, prediction, label)

In [52]:
def keras_nearest_neighbors(X_train_scale_pca, y_train, X_test_scale_pca,y_test, prediction):
  label = "Keras Nearest Neighbors"
  kn_model = KNeighborsClassifier(n_neighbors=3).fit(X_train_scale_pca,y_train)
  prediction = kn_model.predict(X_test_scale_pca)
  matrix_confusion(y_test,prediction,label)



In [53]:
def keras_neural_network(X_train_scale_pca, y_train, X_test_scale_pca,y_test):
  label = "Keras Neural Network"
  
  tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)
  
   # Run the kerastuner search for best hyperparameters
  tuner.search(X_train_scale_pca,y_train,epochs=50,validation_data=(X_test_scale_pca,y_test))

   # Get best model hyperparameters
  best_hyper = tuner.get_best_hyperparameters(1)[0]
  best_hyper.values

   # Evaluate best model against full test data
  best_model = tuner.get_best_models(1)[0]
  model_loss, model_accuracy = best_model.evaluate(X_test_scale_pca,y_test,verbose=2)
  print(label)
  print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [55]:
# Calling the Randomn Forest, Keras Nearest Neighbor with PCA function and passing in the X_train_scale_pca y_train and X_test_scale_pca
random_forest_pca(X_train_scale_pca, y_train, X_test_scale_pca,y_test, prediction)
keras_nearest_neighbors(X_train_scale_pca, y_train, X_test_scale_pca,y_test, prediction)


Random Forest PCA


,Predicted 0,Predicted 1
Actual 0,760,15
Actual 1,131,9


RF Accuracy Score : 0.8404371584699454
RF Classification Report
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       775
           1       0.38      0.06      0.11       140

    accuracy                           0.84       915
   macro avg       0.61      0.52      0.51       915
weighted avg       0.78      0.84      0.79       915

Keras Nearest Neighbors


,Predicted 0,Predicted 1
Actual 0,733,42
Actual 1,124,16


RF Accuracy Score : 0.8185792349726776
RF Classification Report
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       775
           1       0.28      0.11      0.16       140

    accuracy                           0.82       915
   macro avg       0.57      0.53      0.53       915
weighted avg       0.77      0.82      0.79       915



In [ ]:
# Calling the Keras Neural Network with PCA data and passing in the X_train_scale_pca, y_train, X_test_scale_pca, and y_test
keras_neural_network(X_train_scale_pca, y_train, X_test_scale_pca,y_test)

In [ ]:
# Creating a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1).fit(X_train_scale_pca, y_train)

In [ ]:
# Making Prediction using the testing data
prediction = rf_model.predict(X_test_scale_pca)

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, prediction)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, prediction)

In [ ]:
# Displaying results
print("RF Confusion Matrix")
display(cm_df)
print(f"RF Accuracy Score : {acc_score}")
print("RF Classification Report")
print(classification_report(y_test, prediction))

RF Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,760,15
Actual 1,131,9


RF Accuracy Score : 0.8404371584699454
RF Classification Report
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       775
           1       0.38      0.06      0.11       140

    accuracy                           0.84       915
   macro avg       0.61      0.52      0.51       915
weighted avg       0.78      0.84      0.79       915

